In [ ]:
%%bash

params_dir="/workspace/params"

# Exit early if weights are already present
if [ -f "${params_dir}/params_model_5_ptm.npz" ]; then
    echo "AlphaFold2 weights already downloaded at ${params_dir}"
    exit 0
fi

mkdir -p "${params_dir}" 
cd "${params_dir}" 

echo "Downloading AlphaFold2 weights..."
wget --progress=dot:giga https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar 
echo "Extracting weights..."
tar -xf alphafold_params_2022-12-06.tar 
rm alphafold_params_2022-12-06.tar 

if [ -f "${params_dir}/params_model_5_ptm.npz" ]; then
    echo "AlphaFold2 weights successfully set up at ${params_dir}"
else
    echo "AlphaFold2 weights not found after extraction."
    exit 1
fi


In [ ]:
%%bash
# Install PyRosetta in the Bindcraft docker container
# This is done in the container to avoid installing it in the image.
# Before running this cell, make sure you agree to the terms of use:
# https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.PyRosetta.md

docker exec -u root bindcraft_session bash -c "
  source /opt/conda/etc/profile.d/conda.sh && \
  conda activate BindCraft && \
  mkdir -p /app/pyrosetta && \
  cd /app/pyrosetta && \
  mamba install pyrosetta -c https://conda.graylab.jhu.edu -y
  "

In [ ]:
%%bash
version = $(docker --version 2> /dev/null)

if [ -n "$version"]; then
    echo "Docker is installed: $version"
    docker pull yarrowdocker/bindcraft:latest
else
    echo "Docker is not installed. Will install it now."
    pip install docker
fi

In [ ]:
%%bash
target="PDL1.json"

if [ ! -f "workspace/settings_target/$target" ]; then
  echo "Target file $target not found in settings_target directory."
  exit 1
fi

nohup python /app/bindcraft/bindcraft.py \
  --settings /workspace/settings_target/$target \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > /workspace/outputs/bindcraft_log.txt 2>&1 &


In [ ]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload(accept='.json', multiple=False)
display(uploader)


In [ ]:
ps aux | grep bindcraft
tail -n 50 /workspace/outputs/bindcraft_log.txt
